In [1]:
import sys

from sklearn.model_selection import train_test_split

from src.utils import augmentation_random_cut

PWD = '../..'
sys.path.append(PWD)

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import warnings

warnings.simplefilter('ignore')

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks

from IPython.display import display, HTML

from src.plot_utils import plot_sample, plot_history
from src.cfd import CFD
from src.cfd_utils import TIME_STEP, _get_gauss_stats, plot_diff_hist_stats
from src.models import optimal_model_builder


In [2]:
dataset = np.load(PWD + f'/data/dataset.npz', allow_pickle=True)

In [3]:
all_channels_data = dataset['dataset'].flat[0]
all_channels_data.keys()

dict_keys([8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [4]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

# Utils

In [5]:
def plot_difference_hist(y_true, y_pred, channel, method, hist_range=(-2, 2), n_bins=100, return_gauss_stats=True):
    mu, std = plot_diff_hist_stats(y_true, y_pred, show=False, n_bins=n_bins, hist_range=hist_range, hist_label=f'',
                                   plot_gauss=True, return_gauss_stats=return_gauss_stats)

    plt.title(f'{method}. channel={channel}, mean={mu:0.3f}, std={std:0.3f}')
    return std


def train_model(model, X_train, y_train, X_test, y_test, lr=0.01, name='model', train=True, n_epochs=3000, verbose=1,
                batch_size=2048, lr_patience=10, es_patience=200, loss_weights=10_000):
    model.compile(loss='mse', optimizer=optimizers.Adam(lr), loss_weights=loss_weights)

    model_data_dir = f'/data/model_weights/nn_results/channel_experiments/{name}'
    model_callbacks = []
    model_callbacks.append(callbacks.ModelCheckpoint(filepath=PWD + model_data_dir + "/weights", save_best_only=True,
                                                     save_weights_only=True))
    if es_patience is not None:
        model_callbacks.append(callbacks.EarlyStopping(patience=es_patience, min_delta=0.01))
    if lr_patience is not None:
        model_callbacks.append(callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=lr_patience))

    try:
        pd.read_csv(PWD + model_data_dir + '/loss_log.csv')
    except:
        train = True

    if train:
        history = model.fit(X_train, y_train, epochs=n_epochs, verbose=verbose, batch_size=batch_size,
                            validation_data=(X_test, y_test), callbacks=model_callbacks).history
        pd.DataFrame(history).to_csv(PWD + model_data_dir + '/loss_log.csv')

    model.load_weights(PWD + model_data_dir + "/weights")
    history = pd.read_csv(PWD + model_data_dir + '/loss_log.csv')

    return history


def load_model(model, lr=0.01, name='model', loss_weights=None):
    model.compile(loss='mse', optimizer=optimizers.Adam(lr), loss_weights=loss_weights)
    model.load_weights(PWD + f'data/model_weights/nn_results/channel_experiments/{name}/weights')

In [6]:
def pred_and_retrieve_y(model, X):
    Y_heatmap = model.predict(X, batch_size=8192)
    y_pred = np.empty(len(X))
    for i, y in enumerate(Y_heatmap):
        try:
            y_pred[i] = _get_gauss_stats(y, std_0=0.8)
        except:
            print(f'Error calculating {i}')
            y_pred[i] = np.average(np.arange(len(y)), weights=y)
    return y_pred


def compare_model_with_cfd(model, cfd, X, y_true, channel=None, figsize=(12, 4), show=True, y_cfd=None,
                           return_gauss_stats=True, log=True):
    if y_cfd is None:
        y_cfd = np.array([cfd.predict(x) for x in X])
        n_none = list(y_cfd).count(None)
        if n_none > 0:
            print(f'--------------------> CFD returned {n_none} Nones out of {len(y_cfd)} examples')
    y_cfd = y_cfd.astype(np.float64)
    y_model = pred_and_retrieve_y(model, X)

    mask = y_cfd != None
    y_true = y_true[mask]
    y_cfd = y_cfd[mask]
    y_model = y_model[mask]

    if log and channel is not None:
        display(HTML(f'<h4>Channel {channel}</h4>'))

    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    std_cfd = plot_difference_hist(y_true, y_cfd, channel, 'CFD', hist_range=(-6, 6), n_bins=300,
                                   return_gauss_stats=return_gauss_stats)
    plt.subplot(1, 2, 2)
    std_model = plot_difference_hist(y_true, y_model, channel, 'NN', hist_range=(-6, 6), n_bins=300,
                                     return_gauss_stats=return_gauss_stats)

    improvement = (std_cfd - std_model) / std_cfd

    if log:
        print(
            f"CFD: {std_cfd:0.3f}, NN: {std_model:0.3f} ({std_cfd * TIME_STEP:0.3f} ns vs {std_model * TIME_STEP:0.3f} ns)")
        print(f'Improvement: {improvement * 100:0.2f} %')
    if show:
        plt.show()
    else:
        plt.close()
    return improvement, std_cfd * TIME_STEP, std_model * TIME_STEP


cfd = CFD(n_baseline=10, fraction=0.25)

In [7]:
def gaussian_kernel(mu, sigma=0.8, n=48):
    x = np.arange(0, n)
    return np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))


_get_gauss_stats(gaussian_kernel(np.array(1.6)))

1.6

## Trained separately, tested on the same channels

In [8]:
improvements = {}
for channel, (X, y) in list(all_channels_data.items()):
    X_aug, y_aug = augmentation_random_cut(X, y, 16, seed=42, apply=True)
    X_train, X_test, y_train, y_test = train_test_split(X_aug, y_aug, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    Y_heatmap_train = np.array([gaussian_kernel(y) for y in y_train])
    Y_heatmap_test = np.array([gaussian_kernel(y) for y in y_test])

    model = optimal_model_builder()
    history = train_model(model, X_train, Y_heatmap_train, X_test, Y_heatmap_test, name=f'model_separate_{channel}',
                          train=False, n_epochs=3000, verbose=0)

    # plot_history(history, f"Channel {channel}, {len(X)} events", ymax=100, figsize=(8, 4))
    improvement, _, std_model = compare_model_with_cfd(model, cfd, X_test, y_test, channel, show=False)
    improvements[channel] = improvement, std_model

1/1 [==============================] - 2s 2s/step


CFD: 0.453, NN: 0.390 (0.071 ns vs 0.061 ns)
Improvement: 13.88 %
1/1 [==============================] - 0s 348ms/step


CFD: 0.458, NN: 0.413 (0.072 ns vs 0.065 ns)
Improvement: 9.72 %
1/1 [==============================] - 1s 634ms/step


CFD: 0.407, NN: 0.353 (0.064 ns vs 0.055 ns)
Improvement: 13.34 %
1/1 [==============================] - 0s 434ms/step


CFD: 0.423, NN: 0.357 (0.066 ns vs 0.056 ns)
Improvement: 15.58 %
1/1 [==============================] - 1s 768ms/step
Error calculating 5103


...

CFD: 0.709, NN: 0.575 (0.111 ns vs 0.090 ns)
Improvement: 18.83 %
1/1 [==============================] - 0s 343ms/step


CFD: 0.487, NN: 0.386 (0.076 ns vs 0.060 ns)
Improvement: 20.73 %
1/1 [==============================] - 0s 262ms/step


CFD: 0.444, NN: 0.381 (0.069 ns vs 0.060 ns)
Improvement: 14.15 %
1/1 [==============================] - 0s 351ms/step


CFD: 0.468, NN: 0.396 (0.073 ns vs 0.062 ns)
Improvement: 15.38 %
1/1 [==============================] - 0s 271ms/step


CFD: 0.509, NN: 0.429 (0.079 ns vs 0.067 ns)
Improvement: 15.68 %


In [9]:
channels = improvements.keys()
precision_data = [f'{std_model * 1000:0.1f}' for _, std_model in improvements.values()]
improvement_data = [f'{improv * 100:0.0f} %' for improv, _ in improvements.values()]

df = pd.DataFrame({'precision [ps]': precision_data, 'improvement': improvement_data}, index=channels)
df.index.name = 'channel'
df
# df.to_csv('channel_precision_improvements.csv')

,precision [ps],improvement
channel,,
8,60.9,14 %
9,64.6,10 %
10,55.1,13 %
11,55.7,16 %
13,99.1,19 %
14,91.7,22 %
16,53.4,23 %
17,58.1,19 %
18,59.5,14 %
